In [6]:
import rootutils
rootutils.setup_root('.', indicator=".project-root", pythonpath=True)
from src.model.model_module import LitMML, MLMWrapper
from src.model.utils import get_model_and_processor
from omegaconf import OmegaConf
import torch

In [31]:
text_ckpt_path = "/home/data/arena-multimodal/mlm_ckpts/mkqp8hhx/ckpt-epoch=74-loss-val=0.885.ckpt"
image_ckpt_path = "/home/data/arena-multimodal/mlm_ckpts/txva2y48/ckpt-epoch=79-loss-val=1.435.ckpt"

text_ckpt = torch.load(text_ckpt_path, map_location='cpu')
image_ckpt = torch.load(image_ckpt_path, map_location='cpu')

In [32]:
model, processor = get_model_and_processor(config=OmegaConf.load('configs/model/dual_encoder.yaml'))
model = MLMWrapper(model)
lit_model = LitMML.load_from_checkpoint(image_ckpt_path, model=model, processor=processor)
model, processor = lit_model.model, lit_model.processor

/home/phisch/venv_py3.8/py3.8/lib/python3.8/site-packages/lightning/pytorch/utilities/migration/utils.py:56: The loaded checkpoint was produced with Lightning v2.2.3, which is newer than your current Lightning version: v2.2.0.post0


In [49]:
state_dict = text_ckpt['state_dict']
for key in list(state_dict.keys()):
    if not key.startswith("model.basemodel.text"):
        state_dict.pop(key)
print(state_dict.keys())

odict_keys(['model.basemodel.text_model.embeddings.word_embeddings.weight', 'model.basemodel.text_model.embeddings.position_embeddings.weight', 'model.basemodel.text_model.embeddings.token_type_embeddings.weight', 'model.basemodel.text_model.embeddings.LayerNorm.weight', 'model.basemodel.text_model.embeddings.LayerNorm.bias', 'model.basemodel.text_model.encoder.layer.0.attention.self.query.weight', 'model.basemodel.text_model.encoder.layer.0.attention.self.query.bias', 'model.basemodel.text_model.encoder.layer.0.attention.self.key.weight', 'model.basemodel.text_model.encoder.layer.0.attention.self.key.bias', 'model.basemodel.text_model.encoder.layer.0.attention.self.value.weight', 'model.basemodel.text_model.encoder.layer.0.attention.self.value.bias', 'model.basemodel.text_model.encoder.layer.0.attention.output.dense.weight', 'model.basemodel.text_model.encoder.layer.0.attention.output.dense.bias', 'model.basemodel.text_model.encoder.layer.0.attention.output.LayerNorm.weight', 'model.b

In [51]:
model.basemodel.text_projection.load_state_dict({"weight": state_dict.pop("model.basemodel.text_projection.weight")})
model.basemodel.text_model.load_state_dict({k.replace("model.basemodel.text_model.", ""): v for k, v in state_dict.items()})

<All keys matched successfully>

In [5]:
import pickle

with open("/home/phisch/multimodal/test_results/different_modality_weights/3burj4np-zeroshot-results.p", "rb") as f:
    data = pickle.load(f)

data

{'zeroshot_CIFAR10': {'1_templates': {'ConfusionMatrix': array([[  28,  140,    0, 4432,    1,  137,  252,    2,    6,    2],
          [ 233,  178,   18, 3754,  102,  200,  284,   79,   98,   54],
          [  44,  125,    7, 4587,    3,   27,  207,    0,    0,    0],
          [  33,  166,   11, 4437,    4,  199,  139,    5,    1,    5],
          [  56,   59,    7, 4766,    1,   20,   89,    0,    1,    1],
          [ 156,  186,    3, 4362,    9,  174,   76,   18,   11,    5],
          [  14,   13,    7, 4900,    1,    6,   58,    1,    0,    0],
          [   6,   55,    0, 4336,    0,  251,  344,    2,    0,    6],
          [  16,  147,    0, 4454,    0,  163,  169,    9,   35,    7],
          [  66,  131,    6, 3777,  105,  382,  157,  136,  214,   26]]),
   'Top1Accuracy': 0.09892000257968903,
   'Top3Accuracy': 0.3257400095462799,
   'Top5Accuracy': 0.532039999961853}},
 '__config__': {'model': {'model': {'image_encoder_name': 'google/vit-base-patch16-224', 'text_encoder_na